In [115]:
import pandas as pd
from datasets import Dataset
from transformers import TFDistilBertForSequenceClassification, AutoTokenizer
import tensorflow as tf

# Load the dataset from the file
df = pd.read_csv(r'C:\Users\Suman Kar\Downloads\SMSSpamCollection (1).txt', sep='\t', names=['label', 'message'])

# Map labels to 0 for ham and 1 for spam
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Convert pandas dataframe to Hugging Face dataset
dataset = Dataset.from_pandas(df)

In [116]:

# Load DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)

# Split the dataset into train and test sets
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Apply tokenization to the split dataset
tokenized_datasets = split_dataset.map(tokenize_function, batched=True)

# Print the tokenized datasets to verify
print(tokenized_datasets)

Map:   0%|          | 0/4457 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'message', 'input_ids', 'attention_mask'],
        num_rows: 4457
    })
    test: Dataset({
        features: ['label', 'message', 'input_ids', 'attention_mask'],
        num_rows: 1115
    })
})


In [117]:

# Function to convert to TensorFlow dataset
def convert_to_tf_dataset(dataset, batch_size=8):
    dataset.set_format(type="tensorflow", columns=["input_ids", "attention_mask", "label"])

    return dataset.to_tf_dataset(
        columns=["input_ids", "attention_mask"],  # Input features
        label_cols=["label"],  # Labels
        shuffle=True,  # Shuffle the data
        batch_size=batch_size  # Batch size for training
    )

In [118]:

# Convert train and eval datasets to TensorFlow format
train_dataset_tf = convert_to_tf_dataset(tokenized_datasets["train"])
eval_dataset_tf = convert_to_tf_dataset(tokenized_datasets["test"])

# Verify the TensorFlow dataset format
print(train_dataset_tf)
print(eval_dataset_tf)


<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


C:\Users\Suman Kar\AppData\Roaming\Python\Python312\site-packages\datasets\arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [119]:
model=TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [122]:
model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy']

)


In [ ]:
history = model.fit(train_dataset_tf, epochs=3, validation_data=eval_dataset_tf)

Epoch 1/3


